In [1]:
import os

In [2]:
%pwd

'd:\\SL\\End to End Projects\\Diary production Forcasting\\research'

In [3]:
import os

os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dirs: Path
    train_data_path: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    index_column: str
    

In [5]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml, create_directories,save_json

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
         self.config = read_yaml(config_filepath)
         self.params = read_yaml(params_filepath)
         self.schema = read_yaml(schema_filepath)
         print(self.schema)
         create_directories([self.config.artifacts_root])

    def model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.SARIMAX
        schema_target = self.schema.TARGET_COLUMN
        schema_index = self.schema.INDEX_COLUMN
        
        create_directories([config.root_dirs])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dirs=config.root_dirs,
            test_data_path = config.test_data_path,
            train_data_path = config.train_data_path,
            model_path= config.model_path,
            all_params = params,
            metric_file_name=config.metric_file_name,
            target_column=schema_target.name,
            index_column=schema_index.name
        )
        
        return model_evaluation_config

In [7]:
import joblib
import os
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error, mean_squared_log_error


In [8]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def eval_metrics(self,actual,predictions):
        mae=mean_absolute_error(actual, predictions)
        mse=mean_squared_error(actual, predictions)
        mape = np.mean(np.abs(predictions - actual)/np.abs(actual))*100 
        return mae, mse,mape
    
    def save_results(self):
        train_data = pd.read_csv(self.config.train_data_path, parse_dates = [self.config.index_column], index_col = self.config.index_column)
        test_data = pd.read_csv(self.config.test_data_path, parse_dates = [self.config.index_column], index_col = self.config.index_column)
        model = joblib.load(self.config.model_path)
        start = len(train_data)
        end = len(train_data) + len(test_data) - 1
        predictions = model.predict(start = start, end = end, dynamic = False, typ = 'levels').rename('SARIMA Test Predictions') 
        # print(mean_absolute_error(test_data[self.config.target_column], predictions))
        mae, mse,mape = self.eval_metrics(test_data[self.config.target_column],predictions)
        print(self.eval_metrics(test_data[self.config.target_column],predictions))
        # saving metrics as local
        scores = {"MAE":mae,"MSE":mse,"MAPE":mape}
        save_json(path=Path(self.config.metric_file_name),data=scores)
        
    

In [9]:
try:
    config = ConfigurationManager()
    mode_evalution_config = config.model_evaluation_config()
    mode_evalution_config = ModelEvaluation(config=mode_evalution_config)
    mode_evalution_config.save_results()
except Exception as e:
    raise e;

[2024-08-02 23:32:42,460]: yaml file: config\config.yaml loaded successfully:
[2024-08-02 23:32:42,462]: yaml file: params.yaml loaded successfully:
[2024-08-02 23:32:42,464]: yaml file: schema.yaml loaded successfully:
{'COLUMNS': {'Date': 'object', 'Production': 'int64'}, 'INDEX_COLUMN': {'name': 'Date'}, 'TARGET_COLUMN': {'name': 'Production'}}
Directory 'artifacts' created successfully or already exists.
[2024-08-02 23:32:42,465]: Created directory at: artifacts:
Directory 'artifacts/model_evalution' created successfully or already exists.
[2024-08-02 23:32:42,466]: Created directory at: artifacts/model_evalution:
(14.062888849228928, 274.94063634808697, 1.6499054345183708)
[2024-08-02 23:32:42,562]: json file saved at: artifacts\model_evalution\metrics:


c:\Users\sumai\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['typ']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)
